## Crear la Tabla results_movie en la capa "gold"

In [0]:
dbutils.widgets.text("file_date","2024-12-30")
file_date = dbutils.widgets.get("file_date")

In [0]:
spark.sql("""
          CREATE TABLE IF NOT EXISTS movie_gold.results_movie (
              year_release_date INT,
              country_name STRING,
              company_name STRING,
              budget FLOAT,
              revenue FLOAT,
              movie_id INT,
              country_id INT,
              company_id INT,
              created_date DATE,
              updated_date DATE
          )
          USING DELTA
          """)

In [0]:
spark.sql(f"""
          CREATE OR REPLACE TEMP VIEW vw_results_movie
                AS
                SELECT M.year_release_date,C.country_name,PCO.company_name,M.budget,M.revenue,M.movie_id, C.country_id, PCO.company_id
                FROM movie_silver.movies M
                INNER JOIN movie_silver.production_countries PC ON M.movie_id=PC.movie_id
                INNER JOIN movie_silver.countries C ON PC.country_id=C.country_id
                INNER JOIN movie_silver.movies_companies MC ON M.movie_id=MC.movie_id
                INNER JOIN movie_silver.productions_companies PCO ON MC.company_id=PCO.company_id
                WHERE M.file_date='{file_date}'
            """)

In [0]:
%sql
SELECT * FROM vw_results_movie

In [0]:
spark.sql("""
    MERGE INTO movie_gold.results_movie target
    USING vw_results_movie source
    ON (target.movie_id = source.movie_id AND target.country_id=source.country_id AND target.company_id = source.company_id)
    WHEN MATCHED THEN
    UPDATE SET
        target.year_release_date = source.year_release_date,
        target.country_name = source.country_name,
        target.company_name = source.company_name,
        target.budget = source.budget,
        target.revenue = source.revenue,
        target.updated_date = current_timestamp
    WHEN NOT MATCHED
    THEN INSERT (
        year_release_date,
        country_name,
        company_name,
        budget,
        revenue,
        movie_id,
        country_id,
        company_id,
        created_date
    )
    VALUES (
        year_release_date,
        country_name,
        company_name,
        budget,
        revenue,
        movie_id,
        country_id,
        company_id,
        current_timestamp
    )
    """)

In [0]:
%sql
SELECT COUNT(1) FROM vw_results_movie